# Combine all on- and off-shore land and barrier layers
This is an example workflow for setting up the friction and barriers for an
off-shore least cost paths analysis. This incorporates existing land-based costs
and barriers into the final friction and barrier rasters. Note that costs for 
offshore transmission are not actually calculated. Instead, relative frictions 
are assigned to areas that areas that difficult to lay cables in, such as steep 
terrain and shipping lanes. 

Areas that should not be used for transmission are referred to as barriers. Note 
that barriers are not impenetrable within the routing and may actually be used for 
transmission. The barriers are used as a high friction multiplier to strongly discourage 
transmission.  The multiplier can be set as an option in  the JSON config file for the 
`least-cost-xmission` CLI. 

In [ ]:
import logging
from rex.utilities.loggers import init_logger
from reVX.least_cost_xmission.aoswt_utilities import CombineRasters

logger = logging.getLogger(__name__)
init_logger('reVX', log_level="DEBUG")

## General configuration and layers
The primary goal of this process is creating the friction and barrier layers. 
There are many optional steps that can be performed to adjust the final friction
and barriers.

In [ ]:
# Default layer location so we don't have to write the full path for all tiffs.
layer_dir = '/shared-projects/rev/projects/aoswt/data/exclusions'

# Template raster to pull transform, etc. from
template_f ='/shared-projects/rev/projects/aoswt/data/exclusions/adjusted_shipping_lanes.tif'

In [ ]:
# The barrier rasters have one or more values to "bar" in the first position of
# the tuple. Single values should be an int, multiple values should use a list. 
# Any other values in the raster are considered open and not used as a barrier.
# Simple filenames without a path will have the layer_dir path prepended to them.
barrier_files = [
    (1, 'military_ship_shock_boxes.tif'),
    (1, 'artificial_reefs.tif'),
    (1, 'danger_zones_and_restricted_areas.tif'),
    (1, 'usace_placement_areas.tif'),
    (1, 'usace_sand_borrow_areas.tif'),
    (1, 'usace_sad_sand_borrow_areas.tif'),
    (1, 'boem_sand_borrow_areas.tif'),
    (1, 'boem_marine_mineral_areas.tif'),
    (1, 'unexploded_ordnance_locations_100m_buffer.tif'),
    (1, 'dpp_option_areas.tif'),
    (3, 'dod_designations.tif'),
    ([1,2,3,4], 'ocean_disposal_sites.tif'),
    ([1,2,3,4,6], 'conservation_areas.tif'),
    (1, '/shared-projects/rev/projects/aoswt/data/rasters/conmap_sediment_low_medium_high.tif'),
]

# Force include layers (optional). These layers will override the barrier layers. Cells
# with values in the first position of the tuple (int or list) will be forced
# "open" if barricaded by the barrier_files. Cells with any other values will
# be ignored.
forced_inclusion_files = [
    (1, 'boem_wind_planning_areas_03292021.tif'),
    (1, 'boem_wind_leases_06082021.tif'),
    (1, 'aoswt_ad_hoc_forced_inclusions.tif'),
]

# Friction (cost) layers. The tuples consist of a dict with friction values and the
# path to the raster. The dict keys are the values in the raster with dict values being
# the friction to assign. Any values in the raster not listed in the dict are 
# ignored. Note that overlapping frictions are added together in the final friction file. 
friction_files = [
    # ({'cell value in tiff': 'corresponding friction', ...}, 'filename.tif')
    ({1: 1, 2: 10, 3:5}, 'adjusted_shipping_lanes.tif'),
    ({1: 5}, 'abb_gas_pipelines_61m_buffer.tif'),
    ({1: 10}, 'federal_channels.tif'),
    ({1: 10, 2: 5, 3: 0},'/shared-projects/rev/projects/aoswt/data/rasters/conmap_sediment_low_medium_high.tif'),
]

# Bathymetric slope (optional) is used for both friction and as a barrier. Slope configuration can be set
# in CombineRasters.__init__()
slope_file = 'atlantic_coast_slope.tif'

# Bathymetry (optional). Areas greater than a selected depth can be assigned a friction value.
bathy_file = '/shared-projects/rev/projects/weto/fy21/offshore/data/rasters/keepers/bathymetry.tif'

# A land/ocean raster must be used to determine where to use land-based costs and barriers
# versus offshore friction and barriers. This can be created from a vector if needed.
land_shp = '/shared-projects/rev/projects/aoswt/data/shapefiles/coast/gshhs_f_l1_rev.shp'

In [ ]:
cr = CombineRasters(template_f, layer_dir=layer_dir)

## Build Offshore Barrier Layer

In [ ]:
# Build composite barrier layer. 
cr.build_off_shore_barriers(barrier_files, forced_inclusion_files, slope_file=slope_file, 
                            save_tiff=True)

## Minimum Friction Layers (Optional)
One or more minimum friction rasters can be used to assign a minimum friction if desired.
In this example, the land shapefile is being buffered to create a near-shore minimum
friction layer. Other raster(s) could be used for minimum friction. If multiple minimum 
friction layers are specified, and they overlap, the highest minimum friction value is 
used. The minimum friction operation is applied after all other friction layers including 
depth and slope are combined. 

Note that any friction or barriers created in this notebook are ignored on land, where 
the land-based costs and barriers are used instead. 


In [ ]:
CREATE_NEAR_SHORE_FRICTION = True
NEAR_SHORE_FRICTION_FILE = './near_shore_friction.tif'

# The create_land_mask() function is being borrowed here to buffer and rasterize a vector
if CREATE_NEAR_SHORE_FRICTION:
    cr.create_land_mask(land_shp, save_tiff=True,
                        filename=NEAR_SHORE_FRICTION_FILE,
                        buffer_dist=500)  # meters (same units as the template raster)

# Tuple and dict format is the same as the normal friction files. See above.
minimum_friction_files = [
    ({1: 4}, NEAR_SHORE_FRICTION_FILE),
]

## Build Offshore Friction

In [ ]:
# Build composite friction layer. Only friction_files is required.
cr.build_off_shore_friction(friction_files, 
                            slope_file=slope_file, 
                            bathy_file=bathy_file,
                            bathy_depth_cutoff=-200, 
                            bathy_friction=2,
                            minimum_friction_files=minimum_friction_files,
                            save_tiff=True)

## Create H5 File 

In [ ]:
# Existing AOSWT h5 file to pull lat/lng, profile from
ex_aoswt_h5 = '/shared-projects/rev/projects/aoswt/data/exclusions/AOSWT_Exclusions.h5'

# New AOSWT h5 file to write costs and barriers to
aoswt_h5 = './example_aoswt_costs.h5'

# Existing land costs and barrier
land_h5 = '/shared-projects/rev/exclusions/xmission_costs.h5'
land_barrier_layer = 'transmission_barrier'
land_costs_layer = 'tie_line_costs_102MW'

In [ ]:
# Create new AOSWT h5 file. This is time intensive and should only be performed
# once. 
CREATE_NEW_H5_FILE = True
if CREATE_NEW_H5_FILE:
    cr.create_aoswt_h5(ex_aoswt_h5, aoswt_h5, overwrite=True)

In [ ]:
# A land mask tiff is used to determine where to use land-based costs and barriers,
# versus using the offshore friction and barriers. A vector can be converted to a 
# tiff if one is not already available. Note that a poorly chosen land mask and result
# in friction gaps near the shore with no friction. The default land mask file name is 
# set in CombineRasters.LAND_MASK_FNAME.
CREATE_LAND_MASK = True
if CREATE_LAND_MASK:
    cr.create_land_mask(land_shp, save_tiff=True)
else:
    cr.load_land_mask()

## Merge land-based and offshore frictions and barriers

In [ ]:
# Merge barriers and write to h5
cr.merge_os_and_land_barriers(land_h5, land_barrier_layer, aoswt_h5, save_tiff=True)

In [ ]:
# Merge frictions and write to h5. Note that land costs are being scaled down 
# a predetermined amount so that paths can route over short stretches of land.
cr.merge_os_and_land_friction(land_h5, land_costs_layer, aoswt_h5, 
                              land_cost_mult=1/15000, save_tiff=True) 

## Done!
Congrats, you've now created the necessary layers to perform a least cost transmission analysis. 
Other steps that may be required to run the analysis:

- Create the JSON configuration file (includes selected desired SC points).
- Create the POI GeoPackage file. 